**Modulo 5 : Regularización y Dropout**
* Instructor: [Juan Maniglia](https://juanmaniglia.github.io)

# Part 5.2: Uso de K-Fold Cross-validation con Keras

La validación cruzada se puede utilizar para una variedad de propósitos en el modelado predictivo. Éstas incluyen:

* Generación de predicciones fuera de la muestra a partir de una red neuronal
* Estime una buena cantidad de épocas para entrenar una red neuronal (parada temprana)
* Evaluar la efectividad de ciertos hiperparámetros, como funciones de activación, recuentos de neuronas y recuentos de capas

La validación cruzada utiliza varios pliegues y múltiples modelos para proporcionar a cada segmento de datos la oportunidad de servir como conjunto de validación y entrenamiento. La validación cruzada se muestra en la Figura 5.CROSS.

**Figure 5.CROSS: K-Fold Crossvalidation**
![K-Fold Crossvalidation](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_1_kfold.png "K-Fold Crossvalidation")

Es importante señalar que habrá un modelo (red neuronal) para cada pliegue. Para generar predicciones para nuevos datos, que son datos que no están presentes en el conjunto de entrenamiento, las predicciones de los modelos de pliegue se pueden manejar de varias maneras:

* Elija el modelo que tuvo el puntaje de validación más alto como modelo final.
* Preestablecer nuevos datos a los 5 modelos (uno para cada pliegue) y promediar el resultado.
* Vuelva a entrenar un nuevo modelo (usando la misma configuración que la validación cruzada) en todo el conjunto de datos. Entrena para tantas épocas y con la misma estructura de capas ocultas.

En general, prefiero el último enfoque y volveré a entrenar un modelo en todo el conjunto de datos una vez que haya seleccionado los hiperparámetros. Por supuesto, siempre reservaré un conjunto final de reserva para la validación del modelo que no uso en ningún aspecto del proceso de capacitación.

### Regresión vs Clasificación K-Fold Cross-Validation

La regresión y la clasificación se manejan de manera algo diferente con respecto a la validación cruzada. La regresión es el caso más simple en el que simplemente puede dividir el conjunto de datos en K pliegues sin tener en cuenta dónde cae cada elemento. Para la regresión, es mejor que los elementos de datos caigan en los pliegues de la forma más aleatoria posible. También es importante recordar que no todos los pliegues necesariamente tendrán exactamente la misma cantidad de elementos de datos. No siempre es posible que el conjunto de datos se divida uniformemente en K pliegues. Para la validación cruzada de regresión, usaremos la clase Scikit-Learn **KFold**.

La validación cruzada para la clasificación también podría usar el objeto **KFold**; sin embargo, esta técnica no garantizaría que el equilibrio de clases permanezca igual en cada pliegue que en el original. Es muy importante que el equilibrio de las clases en las que se entrenó un modelo siga siendo el mismo (o similar) al conjunto de entrenamiento. Una desviación en esta distribución es una de las cosas más importantes para monitorear después de que un modelo entrenado se haya puesto en uso real. Debido a esto, queremos asegurarnos de que la validación cruzada en sí misma no introduzca un cambio no deseado. Esto se conoce como muestreo estratificado y se logra mediante el uso del objeto de Scikit-Learn **StratifiedKFold** en lugar de **KFold** siempre que utilice la clasificación. En resumen, se deben usar los siguientes dos objetos en Scikit-Learn:

* **KFold** Cuando se trata de un problema de regresión.
* **StratifiedKFold** Cuando se trata de un problema de clasificación.

Las siguientes dos secciones demuestran la validación cruzada con clasificación y regresión.

### Predicciones de regresión fuera de la muestra con validación cruzada K-Fold

El siguiente código entrena el conjunto de datos simple mediante una validación cruzada de 5 veces. El rendimiento esperado de una red neuronal, del tipo entrenado aquí, sería la puntuación de las predicciones generadas fuera de la muestra. Comenzamos preparando un vector de características utilizando el jh-simple-dataset para predecir la edad. Este es un problema de regresión.

In [1]:
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

# Leer el dataset
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generar dummies para 'job'
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generar dummies para 'area'
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Generar dummies para 'product'
df = pd.concat([df,pd.get_dummies(df['product'],prefix="product")],axis=1)
df.drop('product', axis=1, inplace=True)

# Tratar Missing values en income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standarizar rangos
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convertir a numpy - Clasificación
x_columns = df.columns.drop('age').drop('id')
x = df[x_columns].values
y = df['age'].values

Ahora que se ha creado el vector de características, se puede realizar una validación cruzada de 5 veces para generar predicciones fuera de la muestra. Asumiremos 500 épocas y no utilizaremos la detención anticipada. Más adelante veremos cómo podemos estimar un recuento de épocas más óptimo.

In [3]:
EPOCHS=500

In [4]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

# Cross-Validate
kf = KFold(5, shuffle=True, random_state=42) # Uso para KFold in clasificación
oos_y = []
oos_pred = []

fold = 0
for train, test in kf.split(x):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(20, input_dim=x.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=0,
              epochs=EPOCHS)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)    

    # RMSE
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print(f"Score (RMSE): {score}")

# Cree la lista de predicción oos y calcule el error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print(f"Final, score (RMSE): {score}")    
    
# cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
#oosDF.to_csv(filename_write,index=False) # para crear un archivo .csv con el dataframe anterior


Fold #1
Score (RMSE): 0.6360556933488891
Fold #2
Score (RMSE): 0.4982818207521656
Fold #3
Score (RMSE): 0.6298677032008315
Fold #4
Score (RMSE): 0.4894054970459144
Fold #5
Score (RMSE): 1.1284291876751646
Final, score (RMSE): 0.7158847831327483


Como puede ver, el código anterior también informa la cantidad promedio de épocas necesarias. Una técnica común es entrenar luego en todo el conjunto de datos para el número promedio de épocas necesarias.

### Clasificación con Stratified K-Fold Cross-Validation

El siguiente código entrena y ajusta el conjunto de datos jh-simple-dataset con validación cruzada para generar archivos fuera de muestra. También escribe los resultados de fuera de muestra (predicciones en el conjunto de prueba).

Es bueno realizar una validación cruzada estratificada de k-fold con datos de clasificación. Esto asegura que los porcentajes de cada clase permanezcan iguales en todos los pliegues. Para ello, utilice el objeto **StratifiedKFold**, en lugar del objeto **KFold** utilizado en la regresión.

In [5]:
import pandas as pd
from scipy.stats import zscore

# Leer el dataset
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generar dummies para 'job'
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generar dummies para 'area'
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Tratar Missing values en income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standarizar rangos
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convertir a numpy - Clasificación
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product']) # Clasificación
products = dummies.columns
y = dummies.values

Asumiremos 500 épocas y no utilizaremos la detención anticipada. Más adelante veremos cómo podemos estimar un recuento de épocas más óptimo.

In [6]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

# np.argmax(pred,axis=1)
# Cross-validate
# Uso para StratifiedKFold clasificación
kf = StratifiedKFold(5, shuffle=True, random_state=42) 
    
oos_y = []
oos_pred = []
fold = 0

# Más especifico y StratifiedKFold 'for'
for train, test in kf.split(x,df['product']):  
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Oculta 1
    model.add(Dense(25, activation='relu')) # Oculta 2
    model.add(Dense(y.shape[1],activation='softmax')) # Salida
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=0,\
              epochs=EPOCHS)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    # probabilidades brutas a la clase elegida (probabilidad más alta)
    pred = np.argmax(pred,axis=1) 
    oos_pred.append(pred)  

    # fold's accuracy
    y_compare = np.argmax(y_test,axis=1) # para accuracy
    score = metrics.accuracy_score(y_compare, pred)
    print(f"Fold score (accuracy): {score}")

# Crear la lista de predicción oos y calcule el error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # para accuracy

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print(f"Final score (accuracy): {score}")    
    
# cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
#oosDF.to_csv(filename_write,index=False)



Fold #1
Fold score (accuracy): 0.6425
Fold #2
Fold score (accuracy): 0.6675
Fold #3
Fold score (accuracy): 0.675
Fold #4
Fold score (accuracy): 0.67
Fold #5
Fold score (accuracy): 0.6525
Final score (accuracy): 0.6615


### Entrenamiento con una validación cruzada y un conjunto de reserva

Si tiene una cantidad considerable de datos, siempre es valioso reservar un conjunto reservado antes de realizar una validación cruzada. Este conjunto de espera será la evaluación final antes de utilizar su modelo para su uso en el mundo real. La figura 5.HOLDOUT muestra esta división.

**Figure 5.HOLDOUT: Cross Validation and a Holdout Set**
![Cross Validation and a Holdout Set](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_3_hold_train_val.png "Cross Validation and a Holdout Set")

El siguiente programa hace uso de un conjunto reservado y, a continuación, realiza una validación cruzada.

In [7]:
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

# Leer el dataset
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generar dummies para 'job'
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generar dummies para 'area'
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Generar dummies para 'product'
df = pd.concat([df,pd.get_dummies(df['product'],prefix="product")],axis=1)
df.drop('product', axis=1, inplace=True)

# Tratar Missing values en income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standarizar rangos
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convertir a numpy - Clasificación
x_columns = df.columns.drop('age').drop('id')
x = df[x_columns].values
y = df['age'].values

Ahora que los datos han sido preprocesados, estamos listos para construir la red neuronal.

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from sklearn.model_selection import KFold

# Mantenga una retención del 10%
x_main, x_holdout, y_main, y_holdout = train_test_split(    
    x, y, test_size=0.10) 


# Cross-validate
kf = KFold(5)
    
oos_y = []
oos_pred = []
fold = 0
for train, test in kf.split(x_main):        
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x_main[train]
    y_train = y_main[train]
    x_test = x_main[test]
    y_test = y_main[test]
    
    model = Sequential()
    model.add(Dense(20, input_dim=x.shape[1], activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    model.fit(x_train,y_train,validation_data=(x_test,y_test),
              verbose=0,epochs=EPOCHS)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred) 

    # accuracy
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print(f"Fold score (RMSE): {score}")


# Crear la lista de predicción oos y calcule el error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print()
print(f"Cross-validated score (RMSE): {score}")    
    
# Escriba la predicción con validación cruzada (de la última red neuronal)
holdout_pred = model.predict(x_holdout)

score = np.sqrt(metrics.mean_squared_error(holdout_pred,y_holdout))
print(f"Holdout score (RMSE): {score}")    


Fold #1
Fold score (RMSE): 0.9962207754668798
Fold #2
Fold score (RMSE): 0.5487815850145982
Fold #3
Fold score (RMSE): 0.6297824164503693
Fold #4
Fold score (RMSE): 0.9789819627753718
Fold #5
Fold score (RMSE): 0.5684672822848418

Cross-validated score (RMSE): 0.7709479474077178
Holdout score (RMSE): 0.539625763275413
